### Juster-Maker:
Tool allowing to run events in cycle

In [80]:
from pytezos import pytezos
import time
from random import choice
from pprint import pprint
from random import randint

# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [81]:
address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
client = pytezos.using(
    key='../2021-05-27-tests-modelling/test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json'
)
contract = client.contract(address)

In [82]:
transaction = contract.newEvent({
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1000000,
    'betsCloseTime': int(time.time()) + 3600,
    'measurePeriod': 3600,
    'liquidityPercent': 10000,
}).as_transaction()

transaction.json_payload()#.hash()

{'protocol': None,
 'branch': None,
 'contents': [{'kind': 'transaction',
   'source': '',
   'fee': '0',
   'counter': '0',
   'gas_limit': '0',
   'storage_limit': '0',
   'amount': '0',
   'destination': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP',
   'parameters': {'entrypoint': 'newEvent',
    'value': {'prim': 'Pair',
     'args': [{'prim': 'Pair',
       'args': [{'prim': 'Pair',
         'args': [{'string': '2021-07-19T16:32:56Z'}, {'string': 'XTZ-USD'}]},
        {'int': '10000'},
        {'int': '3600'}]},
      {'int': '1000000'}]}}}],
 'signature': None}

In [131]:
import asyncio
from abc import abstractmethod
from asyncio import Queue
import time


async def repeat_until_succeed(
        func, allowed_exceptions=None, max_attempts=10, wait_after_fail=10):
    """ Runs func for multiple times if it was failed with any of the allowed
        exceptions """

    allowed_exceptions = allowed_exceptions or []

    for attempt in range(max_attempts):
        try:
            return await func()

        except Exception as e:
            if type(e) in allowed_exceptions:
                print(f'Ignoring error {type(e)}, {str(e)}')
                await asyncio.sleep(wait_after_fail)
            else:
                raise e


class LoopExecutor:
    """ Executes self.execute each period seconds """

    def __init__(self, period):
        self.period = period
        self.loop = asyncio.get_event_loop()


    def run(self):
        self.task = self.loop.create_task(self.loop_task())


    @abstractmethod
    async def execute(self):
        raise NotImplemented('method execute should be implemented')


    async def loop_task(self):
        while True:
            await self.execute()
            await asyncio.sleep(self.period)


    def stop(self):
        self.task.cancel()


class EventCreationEmitter(LoopExecutor):

    def __init__(self, period, contract, operations_queue, event_params):
        """ ...
            - contract: is pytezos object with Juster contract loaded and
                some key provided
        """

        # TODO: starts_from = 'datetime unixtime', every=seconds instead of period
        super().__init__(period)

        self.contract = contract
        self.operations_queue = operations_queue
        self.event_params = event_params

        self.currency_pair = event_params['currency_pair']


    async def create_event(self):
        event_params = {
            'currencyPair': self.event_params['currency_pair'],
            'targetDynamics': self.event_params['target_dynamics'],
            # TODO: using self.starts_from time in betsCloseTime so this time would
            # be more precise?
            'betsCloseTime': int(time.time()) + self.event_params['bets_period'],
            'measurePeriod': self.event_params['measure_period'],
            'liquidityPercent': self.event_params['liquidity_percent'],
        }

        fees = self.event_params['expiration_fee'] + self.event_params['measure_start_fee']
        transaction = self.contract.newEvent(event_params).with_amount(fees).as_transaction()
        await self.operations_queue.put(transaction)

        # TODO: make logging instead of prints:
        print(f'created newEvent transaction with parameters: {event_params}')


    async def execute(self):
        # TODO: do I need here repeat_until_succeed?
        return await repeat_until_succeed(self.create_event)


class BulkSender(LoopExecutor):
    """ Listens to the queue and packs all new operations to the bulk,
        signs and sends the transaction
    """

    def __init__(self, period, client, operations_queue):
        super().__init__(period)

        self.client = client
        self.operations_queue = operations_queue


    async def sign(self, max_operations=10):
        # TODO: check if ready to sign
        # ? await self.is_ready_to_sign()

        operations = []
        while (not self.operations_queue.empty()
               and (len(operations) < max_operations)):
            operations.append(await self.operations_queue.get())

        try:
            # TODO: logging instead of printing, add .json_payload() info and hahses
            print(f'making bulk of {len(operations)} operations')
            result = self.client.bulk(*operations).autofill().sign()
            print(f'signed, result hash: {result.hash()}')
            # TODO: add inject()
            return result

        except Exception as e:
            print(f'catched {type(e)} in sign: {str(e)}')
            # TODO: here I need to classify error and if it was RPC error: return
            # operations back to the queue, if not: raise this e
            import pdb; pdb.set_trace()
            raise e


    async def is_ready_to_sign(self, sleep_time=90):
        """ Waits while client would be possible to sign transaction
            - simple solution is just timer, better would be to reduce 
                this sleep_time value and somehow ask RPC if key is freed
        """

        return await asyncio.sleep(sleep_time)


    async def execute(self):
        return await repeat_until_succeed(self.sign)


# make queue with transactions
# make bulk manager that takes transactions and send them into blocks

In [133]:
events_params = [
    {
        'currency_pair': currency_pair,
        'target_dynamics': 1_000_000,
        'bets_period': 3600,
        'measure_period': 3600,
        'liquidity_percent': 10_000,
        'expiration_fee': 100_000,
        'measure_start_fee': 100_000
    } for currency_pair in ['XTZ-USD', 'BTC-USD']
]

In [134]:
# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 10

operations_queue = Queue(MAX_QUEUE_SIZE)

# for each event_params EventCreationEmitter is created:
emitters = [
    EventCreationEmitter(
        period=params['bets_period'],
        contract=contract,
        operations_queue=operations_queue,
        event_params=params)
    for params in events_params
]

emitters.append(
    BulkSender(period=60, client=client, operations_queue=operations_queue)
)

In [135]:
for emitter in emitters:
    emitter.run()

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626712959, 'measurePeriod': 3600, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626712959, 'measurePeriod': 3600, 'liquidityPercent': 10000}
making bulk of 2 operations
signed, result hash: op3k5NdZvBfGGqug94RMV8jTbpdgmKihMJRuxQ7zNKzbSLd2vj9


In [136]:
for emitter in emitters:
    emitter.stop()

In [ ]:
task = loop.create_task(emitter.run())

In [3]:


CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}

In [4]:
pytezos


Properties
.key		tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm
.shell		['https://api.tez.ie/rpc/florencenet']
.block_id	head

Helpers
.account()
.activate_account()
.activate_protocol()
.bake_block()
.balance()
.ballot()
.bulk()
.check_message()
.contract()
.delegation()
.double_baking_evidence()
.double_endorsement_evidence()
.endorsement()
.endorsement_with_slot()
.failing_noop()
.now()
.operation()
.operation_group()
.origination()
.proposals()
.reveal()
.seed_nonce_revelation()
.sign_message()
.transaction()
.using()
.wait()

In [ ]:
# using pytezos mock:

# Test event is created:

# Test 